# Objective: Finetune I2I LLM w/ LoRA

In [1]:
!export CUDA_VISIBLE_DEVICES=0,1

In [2]:
import torch
print(torch.cuda.device_count())

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from unsloth import FastLanguageModel
import pprint
import json
from pathlib import Path
import transformers
import os


2
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 3000     # Unsloth auto supports RoPE Scaling internally!
dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.
device = "cuda:1"


if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")
# QLora?

Using eager for attention computation.


In [4]:

# current_metadata = {
#     "site_name": "YongDongIllHighSchool",
#     "user_name": "홍길동",
#     "user_role": "customer", # customer, admin
#     "idu_name": "01_IB5",
#     "idu_mapping": {
#         "01_IB5": ["우리반"],
#         "01_IB7": ["옆반"],
#     },  
#     "modality_mapping": {
#         "roomtemp": ["실내온도"],
#         "settemp": ["설정온도"],
#         "oper": ["전원"],
#     },
#     "current_datetime": "2022-09-30 12:00:00",
# }

current_metadata = json.load(open("metadata.json", "r"))

# <Input Semantic Parsing 조건>
# Temporal: 시간 혹은 시간 범위를 나타내는 정보. 사전정보 참고. 대표적 representation과 Timestamp 형식의 튜플로 표현됨. 아무런 정보가 없을 경우 '지금'으로 표현되지만, 정보가 있는 경우 자제해야 함.
# Spatial: 공간 혹은 공간 범위를 나타내는 정보. 사전정보 참고. 대표적 representation으로 표현됨. 아무런 정보가 없을 경우 '우리반'으로 표현되지만, 정보가 있는 경우 자제해야 함.
# Modality: 정보의 형태 혹은 정보의 속성. 사전정보를 참고해서 제일 유사한 단어로 표현해야 하지만, 사전정보에 없는 경우 추정해야 함. 
# Operation: 정보에 행해지는 조작.


# <절대로 하지 말아야 할 것>
# 이미 명시적으로 시간/공간 정보가 input에 있는데, 없다고 잘못 판단하면 안됨.

common_prompt = """
나는 훌룡한 HVAC 관련 질문 답변을 제공하는 인공지능이다. 사용자의 질문을 받아들여서, 그에 맞는 답변을 제공하는 것이 내 임무이다. 사용자의 질문을 받아들일 때, 다음과 같은 절차를 따라야 한다.
<출력 내용>
1. 'Formalized Input': 사용자의 추상적 질문을 구체화 및 정규화한 결과. 다양한 형태의 질문들을 가장 핵심적이고 근본적인 질문으로 변환한 결과.
2. 'Input Semantic Parsing': Input Semantic Parsing 결과. dict 형태로 구성되며, Temporal, Spatial, Modality, Operation을 가짐.
3. 'Strategy': 질문을 해결하기 위한 전략을 답변 전 고민. Objective: 질문의 근본적 의도 및 답변의 목적. Expected Output: 답변의 예상 결과. Step: 답변을 위한 구체으로 쪼개진 단계.
4. 'Instruction Set': 문제 해결을 위해 나의 실제 행동.

<DDL statement>
CREATE TABLE IF NOT EXISTS data_t
(
    id integer NOT NULL DEFAULT nextval('data_t_id_seq'::regclass),
    idu_id integer,
    roomtemp double precision,
    settemp double precision,
    oper boolean,
    "timestamp" timestamp without time zone NOT NULL
)
    
CREATE TABLE IF NOT EXISTS idu_t
(
    id integer NOT NULL DEFAULT nextval('idu_t_id_seq'::regclass),
    name character varying(50) COLLATE pg_catalog."default",
    metadata character varying(255) COLLATE pg_catalog."default",
    CONSTRAINT idu_t_pkey PRIMARY KEY (id)
)

"""
# common_prompt=""

# Current metadata 데이터셋으로 빼긴해야함

In [5]:
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
model_dir = f"/model/{model_id.replace('/', '-')}"

## Load tokenizer and dataset

In [6]:


# Tokenizer initialization
pretrained_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,  
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # device_map=device,
    cache_dir=f"/model/{model_id}/cache",
    attn_implementation=attn_implementation,
    # local_files_only=True
)


if not os.path.exists(model_dir):
    pretrained_model.save_pretrained(model_dir)
print(f"Model dir: {model_dir}")
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.689 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Bllossom/llama-3.2-Korean-Bllossom-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Model dir: /model/Bllossom-llama-3.2-Korean-Bllossom-3B
Pad Token id: 128004 and Pad Token: <|finetune_right_pad_id|>
EOS Token id: 128009 and EOS Token: <|eot_id|>


In [7]:
from unsloth.chat_templates import get_chat_template

# Initialize the tokenizer with the chat template and mapping

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"
# pretrained_model.config.pad_token_id = tokenizer.eos_token_id

# print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
# print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")
tokenizer_ = get_chat_template(
    tokenizer,
    chat_template = "llama-3", 
    # mapping = {
    #     "role" : "from",
    #     "content" : "value",
    #     "user" : "human", 
    #     "assistant" : "gpt",
    #     }, # ShareGPT style # template에서 role을 from으로 바꿔준다던지..
    map_eos_token = False,        # Maps <|im_end|> to <|eot_id|> instead
) # https://docs.unsloth.ai/basics/chat-templates

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

Model does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Pad Token id: 128004 and Pad Token: <|finetune_right_pad_id|>
EOS Token id: 128009 and EOS Token: <|eot_id|>


In [8]:

# dataset = load_dataset("csv", data_files="./training_dataset.tsv",delimiter="\t")
import re


dataset_tr = json.load(open("training_dataset_v2_directsql_tr.json", "r"))
for i, data in enumerate(dataset_tr):
    dataset_tr[i]["Response"] = str(dataset_tr[i]["Response"])
dataset_tr = Dataset.from_list(dataset_tr)

dataset_ts = json.load(open("training_dataset_v2_directsql_ts.json", "r"))
for i, data in enumerate(dataset_ts):
    dataset_ts[i]["Response"] = str(dataset_ts[i]["Response"])
dataset_ts = Dataset.from_list(dataset_ts)

# dataset = load_dataset("json", data_files="training_dataset_v2_directsql.json")
def formatting_prompts_func(examples):
    convos = []
    # Iterate through each item in the batch (examples are structured as lists of values)
    for input, response in zip(examples['Input'], examples['Response']):
        prompt = {
            "content": common_prompt,
            "role": "system"
        }
        user_input = {
            "content": f"{current_metadata};{input}",
            "role": "user"
        }
        answer = {
            "content": f"{response}",
            "role": "assistant"
        }
        convos.append([prompt, user_input, answer])
        print("Answer length: ", len(response))
        # convos.append([prompt, user_input, answer])
    
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos]
    # remove \n\nCutting Knowledge Date: BLAH BLAH \nToday Date: BLAH BLAH\n\n using regex
    texts = [re.sub(r'(\nCutting Knowledge Date:.*?\nToday Date:.*?\n\n)', '', text) for text in texts]
    
    return {"text": texts}

dataset_tr = dataset_tr.map(formatting_prompts_func, batched=True)
dataset_ts = dataset_ts.map(formatting_prompts_func, batched=True)

print(dataset_tr[0]["text"])
# print(f"seq length: {len(tokenizer.encode(dataset_tr[0]['text']))}")

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Answer length:  1542
Answer length:  958
Answer length:  1103
Answer length:  1462
Answer length:  814
Answer length:  809
Answer length:  1193
Answer length:  1252
Answer length:  1569
Answer length:  899
Answer length:  1319
Answer length:  411
Answer length:  812
Answer length:  416
Answer length:  880
Answer length:  455
Answer length:  291
Answer length:  468
Answer length:  782
Answer length:  1203
Answer length:  1666
Answer length:  1619
Answer length:  281
Answer length:  1302
Answer length:  1302


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Answer length:  809
Answer length:  930
Answer length:  1047
Answer length:  1462
Answer length:  814
Answer length:  809
Answer length:  1193
Answer length:  1252
Answer length:  2103
Answer length:  1329
Answer length:  1061
Answer length:  440
Answer length:  905
Answer length:  447
Answer length:  348
Answer length:  469
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

나는 훌룡한 HVAC 관련 질문 답변을 제공하는 인공지능이다. 사용자의 질문을 받아들여서, 그에 맞는 답변을 제공하는 것이 내 임무이다. 사용자의 질문을 받아들일 때, 다음과 같은 절차를 따라야 한다.
<출력 내용>
1. 'Formalized Input': 사용자의 추상적 질문을 구체화 및 정규화한 결과. 다양한 형태의 질문들을 가장 핵심적이고 근본적인 질문으로 변환한 결과.
2. 'Input Semantic Parsing': Input Semantic Parsing 결과. dict 형태로 구성되며, Temporal, Spatial, Modality, Operation을 가짐.
3. 'Strategy': 질문을 해결하기 위한 전략을 답변 전 고민. Objective: 질문의 근본적 의도 및 답변의 목적. Expected Output: 답변의 예상 결과. Step: 답변을 위한 구체으로 쪼개진 단계.
4. 'Instruction Set': 문제 해결을 위해 나의 실제 행동.

<DDL statement>
CREATE TABLE IF NOT EXISTS data_t
(
    id integer NOT NULL DEFAULT nextval('data_t_id_seq'::regclas

In [9]:
peft_model = FastLanguageModel.get_peft_model(
    pretrained_model,
    r=1110,   # LoRA rank - suggested values: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", 
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=1000,
    lora_dropout=0.0,   # Supports any, but = 0 is optimized
    bias="none",      # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Ideal for long context tuning
    random_state=3407,
    use_rslora=False,   # Disable rank-sensitive LoRA for simpler tasks
    loftq_config=None   # No LoftQ, for standard fine-tuning
)
del pretrained_model

Unsloth 2025.1.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Training config

## Train

In [10]:

torch.cuda.empty_cache()
print(len(dataset_tr))


25


In [11]:
import numpy as np


per_device_train_batch_size, epochs = 4, 40 # 8
gradient_accumulation_steps = int(np.ceil(len(dataset_tr) / per_device_train_batch_size))
print(f"Gradient Accumulation Steps: {gradient_accumulation_steps}")
args = TrainingArguments(
    # num_train_epochs = 1,
    per_device_train_batch_size = per_device_train_batch_size,  # Controls the batch size per device
    per_device_eval_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,  # Accumulates gradients to simulate a larger batch
    max_steps=gradient_accumulation_steps * epochs,
    # 리소스 제약때문에 batch size를 타협해야하는 경우가 발생 -> micro batch size를 줄이고,
 	# accumulated step을 늘려, 적절한 size로 gradient를 구해 weight update
    # https://www.youtube.com/watch?v=ptlmj9Y9iwE
    warmup_steps = gradient_accumulation_steps,
    learning_rate = 1e-4,             # Sets the learning rate for optimization
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,              # Regularization term for preventing overfitting
    lr_scheduler_type = "cosine",  # Sets the learning rate scheduler
    seed = 3407,                        
    output_dir = f"{model_dir}/chkpts",  # Output directory for checkpoints and predictions     
    report_to = "none",              # Enables Weights & Biases (W&B) logging
    logging_steps = gradient_accumulation_steps,                # Sets frequency of logging to W&B
    logging_strategy = "steps",       # Logs metrics at each specified step
    evaluation_strategy="steps",  # enable evaluation during training
    eval_steps=gradient_accumulation_steps,
    eval_accumulation_steps=1, # 낮을수록 eval시 사용하는 메모리 줄어듦
    save_steps=gradient_accumulation_steps,
    save_strategy = "steps",               
    load_best_model_at_end = True,    # Loads the best model at the end
    save_only_model = False           # Saves entire model, not only weights
)

from trl import SFTTrainer

trainer = SFTTrainer(
    model = peft_model,
    processing_class = tokenizer,
    train_dataset = dataset_tr,
    eval_dataset = dataset_ts,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,        # Can make training 5x faster for short sequences.
    args = args,
)






Gradient Accumulation Steps: 7


/usr/local/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/25 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

In [12]:
from unsloth import unsloth_train

trainer_stats = unsloth_train(trainer)  
print(trainer_stats)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25 | Num Epochs = 280
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 7
\        /    Total batch size = 28 | Total steps = 280
 "-____-"     Number of trainable parameters = 1,686,773,760


In [ ]:
# del base_model
# del peft_model

torch.cuda.empty_cache()

# Specify the checkpoint directory
checkpoint_dir = f"{model_dir}/chkpts/checkpoint-{42}"
print(checkpoint_dir)
# Load the tokenizer (ensure it's the same tokenizer used for training)

# Load the base model
peft_model, tokenizer = FastLanguageModel.from_pretrained(
    checkpoint_dir,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="cuda:0",
    # cache_dir="/model",
    attn_implementation=attn_implementation,
    cache_dir=f"/model/{model_id}/cache",
    local_files_only=True
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")
print(f"Padding side: {tokenizer.padding_side}")
# tokenizer.pad_token = tokenizer.eos_token
# 


# Make sure the tokenizer is ready


/model/Bllossom-llama-3.2-Korean-Bllossom-3B/chkpts/checkpoint-42
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.689 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.1.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Pad Token id: 128004 and Pad Token: <|finetune_right_pad_id|>
EOS Token id: 128009 and EOS Token: <|eot_id|>
Padding side: right


In [ ]:
if False:
    # Local saving
    peft_model.save_pretrained("lora_i2i") 
    tokenizer.save_pretrained("lora_i2i")

    # For merging the LoRA adapters with the base model and save the model to 16-bit precision for optimized performance with vLLM, use:
    # # Merge to 16bit
    peft_model.save_pretrained_merged("i2i_merged_16bit", tokenizer, save_method = "merged_16bit",)
    peft_model.save_pretrained_merged("i2i_merged_4bit", tokenizer, save_method = "merged_4bit_forced",)
    # model.push_to_hub_merged("<hf_username/model_name>", tokenizer, save_method = "merged_16bit", token = hf_token)

## Tryout

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
if False:
    # del peft_model
    torch.cuda.empty_cache()
    peft_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "i2i_merged_16bit",        # Trained model either locally or from huggingface
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = False,
        attn_implementation=attn_implementation,
        # device_map=[device],
        local_files_only=True
    )
FastLanguageModel.for_inference(peft_model)  # Enable native 2x faster inference


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=1110, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=1110, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Li

[Instruction(['q', '이번달 우리반 실내온도가 최고인 날짜 알려줘.', '이번달 우리반 실내온도가 최고인 날짜', '값', Semantic(Temporal=[('이번달', '2022-09-01 00:00:00 ~ 2022-09-30 23:59:59')], Spatial=['우리반'], Modality=['실내온도'], Operation=['최고인'], Target=['날짜'])]), Instruction(['r', "예) '이번달 우리반의 실내 온도가 가장 더웠던 날은 2022년 9월 15일입니다.'"])]

In [ ]:
import re, time

def extract_content(text):
    # Define the regex pattern to extract the content
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def run(query):
    start_time = time.time()
    chat = [
        {"role": "system", "content": f"{common_prompt}"},
        # {"role": "metadata", "content": f"{current_metadata}"},
        {"role": "user", "content": f"{current_metadata};{query}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        chat,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = peft_model.generate(
        input_ids = inputs,
        max_new_tokens = max_seq_length,
        use_cache = True,
        pad_token_id = tokenizer.eos_token_id,
        eos_token_id = tokenizer.eos_token_id,
    )
    
    response = tokenizer.batch_decode(outputs)[0]
    # print(response)
    # print("Extracting..")
    parsed_response = extract_content(response)
    pprint.pprint(f"Query: {query}, Time: {time.time() - start_time}")
    print(parsed_response)
    return parsed_response
    
    # text_streamer = transformers.TextStreamer(
    #     tokenizer, 
    # skip_prompt = True
    # )
    # _ = peft_model.generate(
    #     input_ids = inputs, 
    #     streamer = text_streamer, 
    #     max_new_tokens = max_seq_length, 
    #     use_cache = True,
    #     pad_token_id = tokenizer.eos_token_id,
    #     eos_token_id = tokenizer.eos_token_id,
    # )
    

In [ ]:
run("오늘 우리반과 옆반의 평균 온도차이 알려줘")



'Query: 오늘 우리반과 옆반의 평균 온도차이 알려줘, Time: 17.82789659500122'
{'Formalized Input': '오늘 우리반과 옆반의 평균 실내온도 차이 알려줘.', 'Input Semantic Parsing': {'Temporal': {'오늘': '2022-09-30 00:00:00 ~ 2022-09-30 23:59:59'}, 'Spatial': {'우리반': '01_IB5', '옆반': '01_IB7'}, 'Modality': {'실내온도': 'roomtemp'}, 'Operation': ['평균의 차이']}, 'Strategy': {'Objective': '오늘 우리반과 옆반 각각의 평균 실내온도 쿼리 후, 두 값의 차이를 계산함.', 'Expected Outputs': ['우리반(19.3℃)이 옆반(17.0℃)보다 2.3℃ 높습니다.'], 'Step': ['우리반의 오늘 실내온도 쿼리결과 qr_roomtemp_ours 에 저장', '옆반의 오늘 실내온도 쿼리결과 qr_roomtemp_beside 에 저장', '각 반의 실내온도 평균 계산', '각 반의 평균 실내온도와 차이 반환']}, 'Instruction Set': [{'type': 'q', 'run': {'args': {'table_name': 'data_t', 'columns': ['roomtemp'], 'conditions': ["timestamp BETWEEN '2022-09-30 00:00:00' AND '2022-09-30 23:59:59'"],'subquery': "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB5')"}},'result': {'name': 'qr_roomtemp_ours'}}, {'type': 'q', 'run': {'args': {'table_name': 'data_t', 'columns': ['roomtemp'], 'conditions': ["timestamp BETWEEN '2022-09-30

'{\'Formalized Input\': \'오늘 우리반과 옆반의 평균 실내온도 차이 알려줘.\', \'Input Semantic Parsing\': {\'Temporal\': {\'오늘\': \'2022-09-30 00:00:00 ~ 2022-09-30 23:59:59\'}, \'Spatial\': {\'우리반\': \'01_IB5\', \'옆반\': \'01_IB7\'}, \'Modality\': {\'실내온도\': \'roomtemp\'}, \'Operation\': [\'평균의 차이\']}, \'Strategy\': {\'Objective\': \'오늘 우리반과 옆반 각각의 평균 실내온도 쿼리 후, 두 값의 차이를 계산함.\', \'Expected Outputs\': [\'우리반(19.3℃)이 옆반(17.0℃)보다 2.3℃ 높습니다.\'], \'Step\': [\'우리반의 오늘 실내온도 쿼리결과 qr_roomtemp_ours 에 저장\', \'옆반의 오늘 실내온도 쿼리결과 qr_roomtemp_beside 에 저장\', \'각 반의 실내온도 평균 계산\', \'각 반의 평균 실내온도와 차이 반환\']}, \'Instruction Set\': [{\'type\': \'q\', \'run\': {\'args\': {\'table_name\': \'data_t\', \'columns\': [\'roomtemp\'], \'conditions\': ["timestamp BETWEEN \'2022-09-30 00:00:00\' AND \'2022-09-30 23:59:59\'"],\'subquery\': "idu_id = (SELECT id FROM idu_t WHERE name = \'01_IB5\')"}},\'result\': {\'name\': \'qr_roomtemp_ours\'}}, {\'type\': \'q\', \'run\': {\'args\': {\'table_name\': \'data_t\', \'columns\': [\'roomtemp\'], 

In [ ]:

run("오늘 우리반 평균온도 알려줘")

run("어제 옆반 온도 평균 알려줘")
run("현재 우리반 실내온도 알려줘")

'Query: 오늘 우리반 평균온도 알려줘, Time: 10.161221981048584'
{'Formalized Input': '오늘 우리반의 평균 실내온도를 알려줘.', 'Input Semantic Parsing': {'Temporal': {'오늘': '2022-09-30 00:00:00 ~ 2022-09-30 23:59:59'}, 'Spatial': {'우리반': '01_IB5'}, 'Modality': {'실내온도': 'roomtemp'}, 'Operation': ['평균']}, 'Strategy': {'Objective': '오늘 우리반의 실내온도 데이터를 쿼리 후, 평균값 계산.', 'Expected Outputs': ['오늘 우리반의 평균 실내온도는 24℃ 입니다.'], 'Step': ['우리반의 오늘 실내온도 데이터 쿼리결과 qr_roomtemp_today에 저장', 'qr_roomtemp_today에서 roomtemp의 평균값 반환']}, 'Instruction Set': [{'type': 'q', 'run': {'args': {'table_name': 'data_t', 'columns': ['roomtemp'], 'conditions': ["timestamp BETWEEN '2022-09-30 00:00:00' AND '2022-09-30 23:59:59'"],'subquery': "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB5')"}},'result': {'name': 'qr_roomtemp_today'}}, {'type': 'o', 'python_script_to_process': "avg_roomtemp = qr_roomtemp_today['roomtemp'].mean()",'return': ['avg_roomtemp']}, {'type': 'r', 'templates': ['오늘 우리반의 평균 실내온도는 {avg_roomtemp}℃ 입니다.']}]}


KeyboardInterrupt: 

In [ ]:
run("롯데캐슬의 현재 온도 알려줘")
run("10년전 우리반 온도 알려줘")

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Feb 2025

나는 훌룡한 HVAC 관련 질문 답변을 제공하는 인공지능이다. 사용자의 질문을 받아들여서, 그에 맞는 답변을 제공하는 것이 내 임무이다. 사용자의 질문을 받아들일 때, 다음과 같은 절차를 따라야 한다.
<출력 내용>
1. 'Formalized Input': 사용자의 추상적 질문을 구체화 및 정규화한 결과. 다양한 형태의 질문들을 가장 핵심적이고 근본적인 질문으로 변환한 결과.
2. 'Input Semantic Parsing': Input Semantic Parsing 결과. dict 형태로 구성되며, Temporal, Spatial, Modality, Operation을 가짐.
3. 'Strategy': 질문을 해결하기 위한 전략을 답변 전 고민. Objective: 질문의 근본적 의도 및 답변의 목적. Expected Output: 답변의 예상 결과. Step: 답변을 위한 구체으로 쪼개진 단계.
4. 'Instruction Set': 문제 해결을 위해 나의 실제 행동.

<DDL statement>
CREATE TABLE IF NOT EXISTS data_t
(
    id integer NOT NULL DEFAULT nextval('data_t_id_seq'::regclass),
    idu_id integer,
    roomtemp double precision,
    settemp double precision,
    oper boolean,
    "timestamp" timestamp without time zone NOT NULL
)
    
CREATE TABLE IF NOT EXISTS idu_t
(
    id integer NOT NULL DEFAULT nextval('idu_t_id_seq'::regclass),

'{\'Formalized Input\': \'10년 전 우리반의 실내온도를 알려줘.\', \'Input Semantic Parsing\': {\'Temporal\': {\'10년 전\': \'2012-09-30 00:00:00 ~ 2012-09-30 23:59:59\'}, \'Spatial\': {\'우리반\': \'01_IB5\'}, \'Modality\': {\'실내온도\': \'roomtemp\'}, \'Operation\': [\'질문 응답\']}, \'Strategy\': {\'Objective\': \'10년 전 우리반의 실내온도를 조회함.\', \'Expected Outputs\': [\'10년 전 우리반의 실내온도는 24℃였습니다.\'], \'Step\': [\'우리반의 10년 전 실내온도 데이터를 쿼리해 qr_roomtemp에 저장\', \'qr_roomtemp에서 실내온도 반환\']}, \'Instruction Set\': [{\'type\': \'q\', \'run\': {\'args\': {\'table_name\': \'data_t\', \'columns\': [\'roomtemp\'], \'conditions\': ["timestamp BETWEEN \'2012-09-30 00:00:00\' AND \'2012-09-30 23:59:59\'"],\'subquery\': "idu_id = (SELECT id FROM idu_t WHERE name = \'01_IB5\')"}},\'result\': {\'name\': \'qr_roomtemp\'}}, {\'type\': \'o\', \'python_script_to_process\': "roomtemp = qr_roomtemp[\'roomtemp\'].values[0]",\'return\': [\'roomtemp\']}, {\'type\': \'r\', \'templates\': [\'10년 전 우리반의 실내온도는 {roomtemp}℃였습니다.\']}]}'